In [ ]:
import pandas as pd
import numpy as np
import inspect
import sklearn


In [77]:
help(sklearn)
# 나온 출력 내용을 package contents 부분만 복사해서 메모장에 붙여넣고
# 어떤 메소드가 있는지 확인하려면
from sklearn import *
# inspect.getmembers(sklearn.package_name) 또는 dir(패키지명) 으로 확인가능

Help on package sklearn:

NAME
    sklearn

DESCRIPTION
    Machine learning module for Python
    
    sklearn is a Python module integrating classical machine
    learning algorithms in the tightly-knit world of scientific Python
    packages (numpy, scipy, matplotlib).
    
    It aims to provide simple and efficient solutions to learning problems
    that are accessible to everybody and reusable in various contexts:
    machine-learning as a versatile tool for science and engineering.
    
    See http://scikit-learn.org for complete documentation.

PACKAGE CONTENTS
    __check_build (package)
    _build_utils (package)
    _config
    _distributor_init
    _isotonic
    _loss (package)
    base
    calibration
    cluster (package)
    compose (package)
    conftest
    covariance (package)
    cross_decomposition (package)
    datasets (package)
    decomposition (package)
    discriminant_analysis
    dummy
    ensemble (package)
    exceptions
    experimental (package)
    ext

In [ ]:
from sklearn import compose ,linear_model, metrics, model_selection, pipeline, preprocessing, svm 


In [78]:
# 그냥 print(dir(패키지명)) 해도 되는데, 보기힘드니 반복문으로 출력

for x in dir(sklearn.impute ):
    print(x)

KNNImputer
MissingIndicator
SimpleImputer
__all__
__builtins__
__cached__
__doc__
__file__
__loader__
__name__
__package__
__path__
__spec__
_base
_knn
typing


In [323]:
# 위에서 우선 필요해보이는거 전부 import 시키고 시작 (나머지는 메모장에 따로 복붙해놓고 봐요!)
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit, cross_val_predict, cross_val_score, train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder,StandardScaler,MinMaxScaler, LabelEncoder
from sklearn.svm import LinearSVC, SVC, OneClassSVM
from sklearn.ensemble import RandomForestClassifier, VotingClassifier,RandomForestRegressor


In [355]:
X_train_df = pd.read_csv('./datasets/exam_sample/x_train.csv')
X_test_df = pd.read_csv('./datasets/exam_sample/x_test.csv')
y_train_df = pd.read_csv('./datasets/exam_sample/y_train.csv')

In [288]:
print(X_train_df.describe(), X_train_df.info(), X_train_df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  3500 non-null   int64  
 1   총구매액     3500 non-null   int64  
 2   최대구매액    3500 non-null   int64  
 3   환불금액     1205 non-null   float64
 4   주구매상품    3500 non-null   object 
 5   주구매지점    3500 non-null   object 
 6   내점일수     3500 non-null   int64  
 7   내점당구매건수  3500 non-null   float64
 8   주말방문비율   3500 non-null   float64
 9   구매주기     3500 non-null   int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 273.6+ KB
           cust_id          총구매액         최대구매액          환불금액         내점일수  \
count  3500.000000  3.500000e+03  3.500000e+03  1.205000e+03  3500.000000   
mean   1749.500000  9.191925e+07  1.966424e+07  2.407822e+07    19.253714   
std    1010.507298  1.635065e+08  3.199235e+07  4.746453e+07    27.174942   
min       0.000000 -5.242152e+07 -2.992000e+06  5.600000e+03     

In [365]:
product_list = X_train_df['주구매상품'].tolist()+X_test_df['주구매상품'].tolist()
product_le = LabelEncoder()
product_le.fit(product_list)
place_list = X_train_df['주구매지점'].tolist()+X_test_df['주구매지점'].tolist()
place_le = LabelEncoder()
place_le.fit(place_list)

LabelEncoder()

In [366]:
X_train_product = product_le.transform(X_train_df['주구매상품'])
X_test_product = product_le.transform(X_test_df['주구매상품'])
X_train_place = place_le.transform(X_train_df['주구매지점'])
X_test_place = place_le.transform(X_test_df['주구매지점'])


In [338]:
proc_x_train_df = copy.deepcopy(X_train_df)
proc_x_test_df = copy.deepcopy(X_test_df)
proc_x_train_df['주구매상품'] = X_train_product
proc_x_test_df['주구매상품'] = X_test_product
proc_x_train_df['주구매지점'] = X_train_place
proc_x_test_df['주구매지점'] = X_test_place

In [292]:
import copy
combined_df = copy.deepcopy(proc_x_train_df)
combined_df['gender'] = y_train_df['gender']
combined_df = combined_df.drop('cust_id',axis=1)

In [328]:
combined_df.head()

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기,gender
0,68282840,11264000,6860000.0,5,0,19,3.894737,0.527027,17,0
1,2136000,2136000,300000.0,21,19,2,1.500000,0.000000,1,0
2,3197000,1639000,NaN,6,1,2,2.000000,0.000000,1,1
3,16077620,4935000,NaN,5,2,18,2.444444,0.318182,16,1
4,29050000,24000000,NaN,15,8,2,1.500000,0.000000,85,0


In [329]:
combined_df['내점당구매액수'] = combined_df['총구매액']/combined_df['내점일수']

In [332]:
combined_df.corr()['gender'].sort_values(ascending=False)

gender     1.000000
주말방문비율     0.073598
구매주기       0.041450
주구매지점      0.015876
내점당구매액수   -0.030117
주구매상품     -0.038668
내점당구매건수   -0.043917
환불금액      -0.078025
최대구매액     -0.114323
총구매액      -0.150141
내점일수      -0.155000
Name: gender, dtype: float64

In [330]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(combined_df,combined_df['gender']):
    strat_train_set = combined_df.loc[train_index]
    strat_test_set = combined_df.loc[test_index]

In [294]:
target_x_train = copy.deepcopy(strat_train_set).drop('gender',axis=1)
target_y_train  = strat_train_set['gender']
target_x_test = copy.deepcopy(strat_test_set).drop('gender',axis=1)
target_y_test = strat_test_set['gender']

In [295]:
print(strat_train_set.columns)

Index(['총구매액', '최대구매액', '환불금액', '주구매상품', '주구매지점', '내점일수', '내점당구매건수', '주말방문비율',
       '구매주기', 'gender'],
      dtype='object')


In [296]:
original_col = ['총구매액', '최대구매액', '환불금액', '주구매상품', '주구매지점', '내점일수', '내점당구매건수', '주말방문비율',       '구매주기', 'gender']
metric_col = ['총구매액', '최대구매액', '환불금액', '내점일수', '내점당구매건수', '주말방문비율', '구매주기']

In [297]:
metric_pipeline = Pipeline(
    [
        ('fillna', SimpleImputer(strategy='constant', fill_value=0)),
     
    ]
)

In [298]:
full_pipeline = ColumnTransformer(
    [
        ('metric', metric_pipeline, metric_col),
    ]
)

prepared_x_train_df = full_pipeline.fit_transform(target_x_train)
prepared_x_test_df  = full_pipeline.fit_transform(target_x_test)

In [356]:
train_backup= copy.deepcopy(prepared_x_train_df)
test_backup = copy.deepcopy(prepared_x_test_df)

In [ ]:
original_col = ['총구매액', '최대구매액', '환불금액', '주구매상품', '주구매지점', '내점일수', '내점당구매건수', '주말방문비율',       '구매주기', 'gender']

In [364]:
train_backup.[:,[]]

(2800, 7)

In [358]:
prepared_x_train_df = copy.deepcopy(train_backup[['내점일수']])
prepared_x_test_df = copy.deepcopy(test_backup[['내점일수']])

<ipython-input-358-0154e902e743>:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  prepared_x_train_df = copy.deepcopy(train_backup[['내점일수']])


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [299]:
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC(probability=True)

voting_clf = VotingClassifier(
    estimators=[('lr',log_clf),('rnd',rnd_clf),('svm',svm_clf)],voting='soft'
)
voting_clf.fit_transform(prepared_x_train_df,target_y_train)


array([[0.55719365, 0.44280635, 0.19      , 0.81      , 0.55799429,
        0.44200571],
       [0.51211047, 0.48788953, 0.18      , 0.82      , 0.60082389,
        0.39917611],
       [0.54952982, 0.45047018, 0.35      , 0.65      , 0.64271049,
        0.35728951],
       ...,
       [0.52622633, 0.47377367, 0.05      , 0.95      , 0.56111616,
        0.43888384],
       [0.7927503 , 0.2072497 , 0.9       , 0.1       , 0.6642614 ,
        0.3357386 ],
       [0.85783903, 0.14216097, 0.86      , 0.14      , 0.66050293,
        0.33949707]])

In [339]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

seed = 7
model = DecisionTreeRegressor(max_depth=5,random_state=seed)
model.fit(prepared_x_train_df, target_y_train)

 


DecisionTreeRegressor(max_depth=5, random_state=7)

In [350]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16,n_jobs=-1)
rnd_clf.fit(prepared_x_train_df, target_y_train)

RandomForestClassifier(max_leaf_nodes=16, n_estimators=500, n_jobs=-1)

In [351]:
pred_y = rnd_clf.predict(prepared_x_test_df)

In [340]:
pred_y = model.predict(prepared_x_test_df)

In [300]:
pred_y = voting_clf.predict(pre)

In [313]:
log_clf.fit(prepared_x_train_df,target_y_train)
pred_y = log_clf.predict(prepared_x_train_df)

In [306]:
target_y_train.shape

(2800,)

In [347]:
from sklearn.metrics import mean_squared_error

final_mse = mean_squared_error(target_y_test, pred_y)
final_rmse = np.sqrt(final_mse)
final_rmse

0.47962241216398754

In [354]:
roc_auc_score(pred_y, target_y_test)

0.608

In [320]:
log_clf.get_params().keys()

dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'])

In [325]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'n_estimators': [3,10,30], 'max_features': [2,4,6,8]},
    {'bootstrap': [False], 'n_estimators': [3,10], 'max_features': [2,3,4]},
]
forest_reg = RandomForestClassifier()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(prepared_x_train_df, target_y_train)

 "/home/kwon/anaconda3/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 279, in fit
    raise ValueError("max_features must be in (0, n_features]")
ValueError: max_features must be in (0, n_features]

  warnings.warn("Estimator fit failed. The score on this train-test"
/home/kwon/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/kwon/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/kwon/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 386, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "/home/kwon/anaconda3/lib/python3.8/site-packages/joblib/parallel.py", line 1029, in __call__
    if self.dispa

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

gender     1.000000
주말방문비율     0.073598
구매주기       0.041450
주구매지점      0.015876
내점당구매액수   -0.030117
주구매상품     -0.038668
내점당구매건수   -0.043917
환불금액      -0.078025
최대구매액     -0.114323
총구매액      -0.150141
내점일수      -0.155000
Name: gender, dtype: float64

In [317]:
print(inspect.getdoc(GridSearchCV))

Exhaustive search over specified parameter values for an estimator.

Important members are fit, predict.

GridSearchCV implements a "fit" and a "score" method.
It also implements "predict", "predict_proba", "decision_function",
"transform" and "inverse_transform" if they are implemented in the
estimator used.

The parameters of the estimator used to apply these methods are optimized
by cross-validated grid-search over a parameter grid.

Read more in the :ref:`User Guide <grid_search>`.

Parameters
----------
estimator : estimator object.
    This is assumed to implement the scikit-learn estimator interface.
    Either estimator needs to provide a ``score`` function,
    or ``scoring`` must be passed.

param_grid : dict or list of dictionaries
    Dictionary with parameters names (`str`) as keys and lists of
    parameter settings to try as values, or a list of such
    dictionaries, in which case the grids spanned by each dictionary
    in the list are explored. This enables searching 

In [106]:
imputer.fit_transform([X_train_df['환불금액'].values])

array([[6860000.,  300000.,       0., ...,       0.,       0., 5973000.]])

In [103]:
X_train_df['환불금액'].values

array([6860000.,  300000.,      nan, ...,      nan,      nan, 5973000.])

In [98]:
np.reshape(X_train_df['환불금액'])

ValueError: Data must be 1-dimensional